In [ ]:
!pip install -q streamlit
import streamlit as st
import pandas as pd
import numpy as np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.5 MB/s eta 0:00:00


In [ ]:
%%javascript
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}setInterval(ClickConnect,60000)

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install pyngrok==4.1.1
from pyngrok import ngrok
!ngrok authtoken *** # replace the *** with your ngrok authentication token to create the frontend locally
public_url = ngrok.connect(port='8501')
print(public_url)

  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15963 sha256=546b76d643849772b4bf060bf44ccf4f07c8b126039275c00fc4d42ca5485014
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902c91a8
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
http://b909-35-224-37-15.ngrok-free.app


In [ ]:
!ngrok http 8501

Your account is limited to 1 simultaneous ngrok agent session.
You can run multiple tunnels on a single agent session using a configuration file.
To learn more, see https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config/

Active ngrok agent sessions in region 'us':
  - ts_2Z6II8P5ZFfSAu3tAIwCbZiP6lS (35.224.37.15)

ERR_NGROK_108



In [ ]:
!pip install -q gpt-2-simple


  Preparing metadata (setup.py) ... done


In [ ]:
!streamlit run /content/app.py &>/content/logs.txt &

In [ ]:
%%writefile app.py

import streamlit as st
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
import os
shared_drive_path = '/content/drive/Shared drives/MSIS 2534 - NLP Group/Final Project'
os.chdir(shared_drive_path)
import gpt_2_simple as gpt2
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess)

file_path_strat = '/content/drive/Shared drives/MSIS 2534 - NLP Group/Final Project/tokened_n_lemmatized.csv'
df_stratified = pd.read_csv(file_path_strat)
df_stratified_clean = df_stratified.dropna(subset=['processed_reviews'])

def SimilarReviewGenerator(review):
    reviews = [review]
    vectorizer = TfidfVectorizer()
    vectorized_reviews = vectorizer.fit_transform(df_stratified_clean['processed_reviews'])
    vectorized_given_review = vectorizer.transform(reviews)
    cosine_similarities = cosine_similarity(vectorized_reviews, vectorized_given_review)
    top5_review_indices = np.argsort(cosine_similarities[:, 0])[-5:][::-1]
    similar_reviews = []
    for index in top5_review_indices:
        closest_review_details = df_stratified_clean.loc[index, ['app_name', 'review_text']]
        similar_reviews.append({
            'App Name': closest_review_details['app_name'],
            'Review Text': closest_review_details['review_text']
        })
    question = 'Write 1 pro (if there exists a pro) and 1 con (if there exists a con) about the reviews.'
    review = str(review)
    reviewog = re.sub(r'[’‘]', '', review)
    complete = question + ": " + reviewog
    ProConList = gpt2.generate(sess,run_name = 'run1', prefix=complete, include_prefix=True, length=150, return_as_list = True)
    return similar_reviews, ProConList

st.title("Steam Game Reviews Wizard :male_mage:")

with st.form(key='my_form'):
  text_input = st.text_input(label='Write a game review without mentioning the name of the game:')
  submit_button = st.form_submit_button(label='Submit')

if submit_button and text_input:
  similar_reviews, ProConList = SimilarReviewGenerator(text_input)

  st.header("Pro & Con:", divider='rainbow')
  st.write(ProConList[0].replace("Write 1 pro (if there exists a pro) and 1 con (if there exists a con) about the reviews.: ", ""))

  st.header("The Top 5 Most Similar Games:", divider='rainbow')
  for i, review in enumerate(similar_reviews):
    st.subheader(f"\nClosest Game #{i + 1}:")
    st.write("**Game Name:**", review['App Name'])
    st.write("**Review:**", review['Review Text'])

Writing app.py


In [ ]:
#!streamlit run /content/app.py &>/content/logs.txt &

In [ ]:
#!pip install -q gpt-2-simple
#import gpt_2_simple as gpt2
#from datetime import datetime
#from google.colab import files
#import os
#shared_drive_path = '/content/drive/Shared drives/MSIS 2534 - NLP Group/Final Project'
#os.chdir(shared_drive_path)
#import gpt_2_simple as gpt2
#sess = gpt2.start_tf_sess()
#gpt2.load_gpt2(sess)

In [ ]:
#import re
#def GenerateResponse(review):
#  question = 'Write 1 pro (if there exists a pro) and 1 con (if there exists a con) about the reviews.'
#  review = str(review)
#  reviewog = re.sub(r'[’‘]', '', review)
#  complete = question + ": " + reviewog
#  gpt2.generate(sess,run_name = 'run1', prefix=complete, include_prefix=True, length=125)


In [ ]:
#review = 'Not having steam cloud saves is the only thing wrong with this. Having to restart every time you want to play on a new device sucks!'
#GenerateResponse(review)